In [1]:
import pandas as pd
import datetime

from bokeh.models import BasicTicker, PrintfTickFormatter, NumeralTickFormatter
from bokeh.plotting import figure, show, output_file

from bokeh.transform import linear_cmap
from bokeh.palettes import Turbo256

from math import pi

from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2

def data_cleaning(df_path):
    
    # assigned data type datetime to the column dt(date)
    df = pd.read_csv(df_path)
    df['dt'] = pd.to_datetime(df['dt'])
    # removing NaN values and resetting the index from 0
    df1 = df.dropna(inplace=False)
    df1.reset_index(drop=True, inplace=True)

    # replacing the symbols to string
    #df1.replace({'Åland':'Åland Islands'}, regex=True) - gave warning for inplace
    df1.loc[df1['Country'] == 'Åland', 'Country'] = 'Åland Islands'
    
    return df1

df1 = data_cleaning("data/GlobalLandTemperaturesByCountry.csv")

In [2]:
continents = {
    'NA': 'North America',
    'SA': 'South America',
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'EU': 'Europe'
}

def get_continent(country_name):
    try:
        alpha2 = country_name_to_country_alpha2(country_name)
        continent_code = country_alpha2_to_continent_code(alpha2)
        continent_name = continents.get(continent_code, 'Unknown')
        return continent_name
    except KeyError:
        return 'Invalid'

df1['Continent'] = df1['Country'].apply(get_continent)
df1 = df1[df1['Continent'] != 'Invalid']

In [3]:
df1 = df1.assign(Year = df1['dt'].dt.year)
df1 = df1.reindex(columns=['Year','dt','Continent','Country','AverageTemperature'])
df1 = df1.drop(columns=['dt','Country'], inplace=False, axis=1)

In [4]:
df1['Year'] = df1['Year'].astype(str)

In [5]:
df1 = df1.pivot_table(index='Year', columns='Continent', values='AverageTemperature') 
df1 = df1.dropna()
df1 = df1.round(2)

In [6]:
years = list(df1.index)
continents = list(df1.columns)
data = pd.DataFrame(df1.stack(), columns=['Temp']).reset_index()

# Tools and colors
colors = Turbo256
TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

In [7]:
p = figure(title=f"Continental Land Temperature for the years ({years[0]} to {years[-1]})",
           x_range=years, y_range=continents,
           x_axis_location="above", width=1100, height=500,
           sizing_mode="stretch_width",
           tools=TOOLS, toolbar_location='below',
           tooltips=[('Year', '@Year'), ('Temp', '@Temp°C')])

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.yaxis.major_label_text_font_size = "10px"
p.xaxis.major_label_text_font_size = "8px"
p.axis.major_label_standoff = 0
p.axis.major_label_text_color = "#000000"
p.xaxis.major_label_orientation = pi/3

r = p.rect(x="Year", y="Continent", width=1, height=1, source=data,
           fill_color=linear_cmap('Temp', colors, low=data.Temp.min(), high=data.Temp.max()),
           line_color=None)

p.add_layout(r.construct_color_bar(
    major_label_text_font_size="8px",
    major_label_text_color="#000000",
    ticker=BasicTicker(desired_num_ticks=len(colors)),
    formatter=PrintfTickFormatter(format="%.2f°C"),
    label_standoff=6,
    border_line_color=None,
    padding=5,
), 'right')

In [8]:
#output_file("main.html")

In [9]:
show(p)